<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
nlp = spacy.load('en_core_web_lg')

C:\Users\navo1\Anaconda3\envs\U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
def tokenize(doc):
    tokenized = [token for token in simple_preprocess(doc) if token not in STOPWORDS]
    return tokenized

In [5]:
# Create tokenized column
yelp['tokenized'] = yelp['text'].apply(lambda x: tokenize(x))
yelp['tokenized'].head()

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
Name: tokenized, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english',min_df=0.01, max_df=0.98, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
sparse = tfidf.fit_transform(yelp['text'])

# Convert sparse matrix to dense matrix and get column names
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

dtm.head()

,able,absolutely,actually,add,added,affordable,afternoon,ago,ahead,air,...,wow,wrong,year,years,years ago,yelp,yes,young,yum,yummy
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.125895,0.0,0.0,0.0,0.0,0.158684,0.0,0.0
3,0.0,0.0,0.245353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [13]:
from sklearn.neighbors import NearestNeighbors

In [14]:
# fit nearest neighbors model with the dataframe
nn  = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [16]:
# create fake review
fake_review = ["This restaurant is my FAV, service is alright,watch out the bathroom is dirty"]

# transform fake review text
transformed_text = tfidf.transform(fake_review)

# apply model to the fake review text
nn.kneighbors(transformed_text.todense())

(array([[1.        , 1.        , 1.        , 1.        , 1.05937221,
         1.06440355, 1.09008816, 1.09531777, 1.12170211, 1.1250151 ]]),
 array([[ 469, 6204, 6311, 9889, 4242, 3735, 7187, 5726, 3780, 8138]],
       dtype=int64))

In [18]:
indicies = [469, 6204, 6311, 9889, 4242, 3735, 7187, 5726, 3780, 8138]
top_ten = fake_review

for index in indicies:
    review = yelp['text'][index]
    print(review, '\n')

O  o  thenk 6nnn  .b  cgv  xx TV cvg  9 nvehxcfvvv3c nb b  c  y4  nb and the vghvhridd h 0d  c       v   3,  vv  4  ruddy 

旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。 

天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來 

Bon massage.  Spa propre organisé manque juste le stationnement prix abordable en promotion personnels respectueux 

I love the waffle house for their speedy service,  this one didn't offer such service. The place was empty but pretty muchb all the tables were dirty......had dirty dishes on them. The staff went about their business behind the counter in no rush to clean the tables. I sat there for almost 20 minutes waiting. No one seemed to care. No one asked me if i needed anything or why I was sitting there etc. The floors were pretty dirty too. Food and 

### The pattern seems to be that the staff aren't great and how dirty everything is. I can't really translate the japanese and chinese.

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
X = yelp['text']
y = yelp['stars']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Create pipeline components
vect = TfidfVectorizer(tokenizer=tokenize, stop_words='english', min_df=0.01, max_df=0.98, ngram_range=(1,2))
clf = SGDClassifier(random_state=42)

# Define the pipeline
pipe = Pipeline([('vect', vect), ('clf', clf)])

# Define grid-search parameters
parameters = {
    'clf__loss': ['hinge', 'log'],
    'clf__alpha': [1e-4, 1e-3, 1e-2],
    'clf__tol': [1e-4, 1e-3, 1e-2]
}

In [26]:
grid_search = GridSearchCV(pipe, parameters, cv=6, n_jobs=4, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 6 folds for each of 18 candidates, totalling 108 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   47.5s
[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:  1.9min finished


GridSearchCV(cv=6, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.98, max_features=None, min_df=0.01,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=Tr...dom_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'clf__loss': ['hinge', 'log'], 'clf__alpha': [0.0001, 0.001, 0.01], 'clf__tol': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [27]:
grid_search.best_score_

0.61425

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [32]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim import corpora

Learn the vocubalary of the yelp data:

In [34]:
def review(df, tokens):

    assert tokens in df.columns, f"{tokens} does not exist!"
    
    for index, row in df.iterrows():
        token = row[tokens]
        yield token

In [36]:
id2word = corpora.Dictionary(review(yelp, 'tokenized'))

Create a bag of words representation of the entire corpus

In [37]:
corpus = [id2word.doc2bow(rev) for rev in review(yelp, 'tokenized')]

Your LDA model should be ready for estimation: 

In [38]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [39]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\navo1\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.004200 -0.002815       1        1  10.582418
0      0.000027  0.002832       2        1  10.513941
1     -0.001084 -0.003934       3        1  10.128036
5     -0.001323 -0.001246       4        1  10.001731
4      0.003844 -0.002265       5        1   9.963877
2     -0.003761  0.002760       6        1   9.888256
7      0.001550  0.001963       7        1   9.880998
9     -0.000243  0.003187       8        1   9.869607
3      0.002142 -0.000525       9        1   9.606489
6      0.003048  0.000043      10        1   9.564647, topic_info=     Category         Freq        Term        Total  loglift  logprob
40    Default  4457.000000        good  4457.000000  30.0000  30.0000
133   Default  4561.000000       place  4561.000000  29.0000  29.0000
144   Default  4754.000000        food  4754.000000  28.0000  28.0000
208   Default  4129.000000       great  4129.000000  27.0000  27.0000
34    Default  3305.000000        time  3305.000000  26.0000  26.0000
387   Default  1450.000000       order  1450.000000  25.0000  25.0000
75    Default  1987.000000         got  1987.000000  24.0000  24.0000
426   Default  1116.000000      better  1116.000000  23.0000  23.0000
92    Default  1689.000000        nice  1689.000000  22.0000  22.0000
651   Default  1332.000000  restaurant  1332.000000  21.0000  21.0000
45    Default  3506.000000     service  3506.000000  20.0000  20.0000
18    Default  3329.000000        like  3329.000000  19.0000  19.0000
246   Default  1056.000000       right  1056.000000  18.0000  18.0000
235   Default  1071.000000         way  1071.000000  17.0000  17.0000
192   Default  1314.000000     amazing  1314.000000  16.0000  16.0000
478   Default  1175.000000        know  1175.000000  15.0000  15.0000
735   Default  1589.000000        love  1589.000000  14.0000  14.0000
111   Default   826.000000       times   826.000000  13.0000  13.0000
440   Default  1167.000000         day  1167.000000  12.0000  12.0000
358   Default  1395.000000  definitely  1395.000000  11.0000  11.0000
421   Default   808.000000       asked   808.000000  10.0000  10.0000
1277  Default  1344.000000     chicken  1344.000000   9.0000   9.0000
1266  Default  1487.000000     ordered  1487.000000   8.0000   8.0000
184   Default  1123.000000   recommend  1123.000000   7.0000   7.0000
157   Default   873.000000       price   873.000000   6.0000   6.0000
535   Default   907.000000        work   907.000000   5.0000   5.0000
205   Default   447.000000         fun   447.000000   4.0000   4.0000
1416  Default   452.000000   breakfast   452.000000   3.0000   3.0000
1465  Default   477.000000      flavor   477.000000   2.0000   2.0000
165   Default   843.000000       think   843.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1242  Topic10    47.640591        stop   353.288971   0.3435  -6.8718
349   Topic10    96.735786      cheese   796.212341   0.2392  -6.1635
165   Topic10   101.721336       think   843.895142   0.2313  -6.1133
75    Topic10   211.329514         got  1987.358765   0.1060  -5.3821
1570  Topic10    71.724510       fries   574.306152   0.2668  -6.4627
94    Topic10   108.900658       night   940.472046   0.1912  -6.0451
208   Topic10   376.271423       great  4129.663086  -0.0485  -4.8052
37    Topic10   155.105637        came  1440.717773   0.1183  -5.6914
1143  Topic10    98.177567    location   840.976746   0.1993  -6.1487
503   Topic10    78.328346      prices   639.070679   0.2480  -6.3746
730   Topic10    90.286850        home   774.851685   0.1974  -6.2325
117   Topic10   143.378952        went  1433.443604   0.0448  -5.7700
137   Topic10   106.886124         bar   982.899536   0.1284  -6.0637
122   Topic10   164.737823        best  1751.240356  -0.0166  -5.6312
133   Topic10   327.282867       place  4561.879883  -0.2876  -4.9447
40    Topic10   319

In [40]:
import squarify
import matplotlib.pyplot as plt

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)